**CAPTURAR OS ROSTOS DETECTADOS**
---



In [ ]:
from IPython.display import Image, display, Javascript
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.output import eval_js # Necessário para take_photo
from base64 import b64decode # Necessário para take_photo

# Certifique-se de que a função take_photo da Célula 4cPh5Yg_7n-P foi executada antes desta.

try:
  # Chama a função para tirar a foto. A função take_photo exibe o botão de captura.
  # A função take_photo está definida na célula 4cPh5Yg_7n-P
  filename = take_photo()
  print('Foto salva como {}'.format(filename))

  # Carregar a imagem salva usando OpenCV
  img = cv2.imread(filename)

  if img is None:
      print("Erro: Não foi possível carregar a imagem capturada.")
  else:
      # Crie uma cópia da imagem original para desenhar os retângulos
      img_com_deteccao = img.copy()

      # Converter a imagem para escala de cinza para a detecção
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

      # Carregar o classificador Haar Cascade para detecção de rostos frontais
      # Certifique-se de que o arquivo haarcascade_frontalface_default.xml foi baixado (célula anterior)
      face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

      if face_cascade.empty():
          print("Erro: Não foi possível carregar o arquivo haarcascade_frontalface_default.xml.")
          print("Verifique se o OpenCV foi instalado corretamente ou se o arquivo existe.")
          # Define faces como lista vazia para evitar erro posterior
          faces = []
      else:
          # Detectar rostos na imagem (pode ajustar os parâmetros)
          faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

          num_pessoas = len(faces)
          print(f"Número de rostos detectados: {num_pessoas}")

          # Desenhar retângulos ao redor dos rostos detectados na cópia da imagem
          # e adicionar um número para identificação
          cor_retangulo = (0, 255, 0) # Verde em BGR
          espessura_retangulo = 2
          fonte_numero = cv2.FONT_HERSHEY_SIMPLEX
          escala_numero = 0.7
          cor_numero = (255, 0, 0) # Vermelho em BGR
          espessura_numero = 2

          for i, (x, y, w, h) in enumerate(faces):
              # Desenha o retângulo
              cv2.rectangle(img_com_deteccao, (x, y), (x+w, y+h), cor_retangulo, espessura_retangulo)
              # Adiciona o número da pessoa (começando do 1)
              cv2.putText(img_com_deteccao, str(i + 1), (x, y - 10), fonte_numero, escala_numero, cor_numero, espessura_numero, cv2.LINE_AA)

          # Converter a imagem com detecção para RGB para exibição
          img_com_deteccao_rgb = cv2.cvtColor(img_com_deteccao, cv2.COLOR_BGR2RGB)

          # Exibir a imagem com os retângulos para que o usuário possa identificar as pessoas
          plt.imshow(img_com_deteccao_rgb)
          plt.title("Rostos Detectados (Identificado pelo número)")
          plt.axis('off')
          plt.show()

          # Armazena o número de pessoas detectadas e os dados dos rostos para a próxima célula
          # Variáveis como num_pessoas e faces estarão disponíveis no kernel
          # filename também estará disponível

except Exception as err:
  print(f"Ocorreu um erro na Célula 1: {str(err)}")
  # Define faces como lista vazia em caso de erro para evitar erro posterior
  faces = []
  num_pessoas = 0
  filename = None # Define filename como None em caso de erro

**INSERIR O NOME DOS INTEGRANTES**

In [ ]:
from IPython.display import Image, display, Javascript
from google.colab.output import eval_js
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Verifica se as variáveis da Célula 1 estão disponíveis
if 'filename' not in locals() or filename is None:
    print("Erro: A Célula 1 não foi executada ou houve um erro na captura da foto.")
elif 'faces' not in locals():
     print("Erro: A Célula 1 não detectou rostos. Execute a Célula 1 primeiro.")
else:
    try:
        # --- Solicitar nomes ao usuário com base nos rostos detectados ---
        nomes_equipe = []
        # Solicita nomes apenas para o número de pessoas detectadas na Célula 1, até o limite de 3.
        num_nomes_solicitados = min(len(faces), 3) # Usa len(faces) para garantir que corresponde ao que foi detectado

        if num_nomes_solicitados > 0:
            print(f"\nPor favor, insira os nomes para {num_nomes_solicitados} rosto(s) detectado(s) (veja a imagem na Célula 1 para identificar pelo número):")
            js_input = '''
                async function getNamesForFaces(num) {
                    let names = [];
                    for (let i = 0; i < num; i++) {
                        let name = prompt("Digite o nome para a Pessoa " + (i + 1) + ":");
                        if (name !== null) {
                            names.push(name);
                        } else {
                            // Se o usuário cancelar, paramos de solicitar nomes
                            break;
                        }
                    }
                    return names;
                }
            '''
            display(Javascript(js_input))
            nomes_equipe = eval_js(f'getNamesForFaces({num_nomes_solicitados})')
            print(f"Nomes inseridos: {nomes_equipe}")
        else:
            print("Nenhum rosto detectado na Célula 1. Nenhum nome será adicionado.")


        # --- Adicionar nomes com OpenCV na imagem original ---
        # Carrega a imagem original novamente para adicionar os nomes
        img = cv2.imread(filename)
        if img is None:
             print("Erro: Não foi possível recarregar a imagem original.")
        else:
            # Usamos os nomes fornecidos pelo usuário, limitados pelo número de rostos detectados (até 3)
            nomes_para_adicionar = nomes_equipe[:min(len(faces), 3)] # Usa len(faces) novamente

            # Adiciona os nomes na imagem original, posicionando-os perto dos rostos detectados
            img_final = img.copy() # Trabalhe em uma nova cópia para adicionar os nomes
            img_final_rgb = cv2.cvtColor(img_final, cv2.COLOR_BGR2RGB)

            for i in range(len(nomes_para_adicionar)):
                if i < len(faces): # Garante que temos um rosto detectado correspondente
                    (x, y, w, h) = faces[i]
                    # Posição para o texto: um pouco acima do retângulo do rosto
                    posicao = (x, y - 10)

                    texto = nomes_para_adicionar[i]
                    fonte = cv2.FONT_HERSHEY_SIMPLEX
                    escala = 0.8 # Escala um pouco menor para caber melhor
                    cor = (255, 0, 0)  # Cor em BGR (Vermelho)
                    espessura = 2
                    tipo_linha = cv2.LINE_AA

                    cv2.putText(img_final_rgb, texto, posicao, fonte, escala, cor, espessura, tipo_linha)
                else:
                    # Isso não deve acontecer com a lógica de min(len(faces), 3), mas por segurança
                    print(f"Aviso: Nome '{nomes_para_adicionar[i]}' não tem rosto correspondente detectado.")


            # Exibir a imagem final com os nomes adicionados usando Matplotlib
            if len(nomes_para_adicionar) > 0: # Só exibe a imagem final se nomes foram adicionados
                plt.imshow(img_final_rgb)
                plt.title("Foto da Equipe com Nomes Adicionados")
                plt.axis('off') # Oculta os eixos
                plt.show()

                # Salvar a imagem modificada com o nome especificado
                cv2.imwrite('foto_nome_integrantes_equipe.png', cv2.cvtColor(img_final_rgb, cv2.COLOR_RGB2BGR))
                print("Imagem modificada salva como 'foto_nome_integrantes_equipe.png'")

            elif len(faces) > 0:
                 print("Nomes não foram inseridos para os rostos detectados.")
            # Se 0 pessoas detectadas, a mensagem já foi exibida na Célula 1.


    except Exception as err:
        print(f"Ocorreu um erro na Célula 2: {str(err)}")